# Machine Learning Project - Credit Modelling

This project will focus on credit modelling, a well-known data science problem that focuses on modeling a borrower's credit risk. We'll be working with financial lending data from Lending Club. Lending Club is a marketplace for personal loans that matches borrowers who are seeking a loan with investors looking to lend money and make a return.

Each borrower completes a comprehensive application, providing their past financial history, the reason for the loan, and more. Lending Club evaluates each borrower's credit score using past historical data and their own data science process to assign an interest rate to the borrower. 

A higher interest rate means that the borrower is a risk and more unlikely to pay back the loan. While a lower interest rate means that the borrower has a good credit history and is more likely to pay back the loan. The interest rates range from 5.32% all the way to 30.99% and each borrower is given a grade according to the interest rate they were assigned. If the borrower accepts the interest rate, then the loan is listed on the Lending Club marketplace.

If a loan is fully paid off on time, the investors make a return which corresponds to the interest rate the borrower had to pay in addition to the requested amount. Many loans aren't completely paid off on time and some borrowers default on the loan.

While Lending Club has to be extremely savvy and rigorous with their credit modelling, investors on Lending Club need to be equally as savvy about determining which loans are more likely to be paid off. If investors believe the borrower can pay back the loan, even if he or she has a weak financial history, then investors can make more money through the larger additional amount the borrower has to pay.

Most investors use a portfolio strategy to invest small amounts in many loans, with healthy mixes of low, medium, and interest loans. In this project, we'll focus on the mindset of a conservative investor who only wants to invest in the loans that have a good chance of being paid off on time. To do that, we'll need to first understand the features in the dataset and then experiment with building machine learning models that reliably predict if a loan will be paid off or not.

## The Data

Lending Club used to release data for all of the approved and declined loan applications periodically on their website.

[The dataset that is used](https://www.kaggle.com/datasets/samaxtech/lending-club-20072011-data) contains the features of the approved loans from 2007 to 2011.

[There is also a data dictionary (in XLS format)](https://docs.google.com/spreadsheets/d/191B2yJ4H1ZPXq0_ByhUgWMFZOYem5jFz0Y3by_7YBY4/edit#gid=2081333097) which contains information on the different column names towards the bottom of the page.

The LoanStats sheet describes the approved loans datasets and the RejectStats describes the rejected loans datasets. Since rejected applications don't appear on the Lending Club marketplace and aren't available for investment, we'll be focusing on approved loans.

The approved loans datasets contain information on current loans, completed loans, and defaulted loans. Let's now define the problem statement for this machine learning project:

Can we build a machine learning model that can accurately predict if a borrower will pay off their loan on time or not?
Before we can start doing machine learning, we need to define what features we want to use and which column represents the target column we want to predict. 

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 100)

loans_2007 = pd.read_csv('loans_2007.csv')

print(loans_2007.iloc[0])
print(loans_2007.shape[1])

id                                1077501
member_id                      1.2966e+06
loan_amnt                            5000
funded_amnt                          5000
funded_amnt_inv                      4975
term                            36 months
int_rate                           10.65%
installment                        162.87
grade                                   B
sub_grade                              B2
emp_title                             NaN
emp_length                      10+ years
home_ownership                       RENT
annual_inc                          24000
verification_status              Verified
issue_d                          Dec-2011
loan_status                    Fully Paid
pymnt_plan                              n
purpose                       credit_card
title                            Computer
zip_code                            860xx
addr_state                             AZ
dti                                 27.65
delinq_2yrs                       

C:\Users\adrig\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Data Cleaning

Since there are 54 columns, three groups of 18 columns are explored seperately to make it less cumbersome. 

The columns that meet the following criteria should be dropped:

- disclose information from the future (after the loan has already been funded)
- don't affect a borrower's ability to pay back a loan (e.g. a randomly generated ID value by Lending Club)
- need to be cleaned up and are formatted poorly
- require more data or a lot of processing to turn into a useful feature
- contain redundant information  

### Columns group 1 of 3

The description of each column can be read in the following table:

| name                | dtype   | first value | description                                                                                                                       |
|---------------------|---------|-------------|-----------------------------------------------------------------------------------------------------------------------------------|
| id                  | object  | 1077501     | A unique LC assigned ID for the loan listing.                                                                                     |
| member_id           | float64 | 1.2966e+06  | A unique LC assigned Id for the borrower member.                                                                                  |
| loan_amnt           | float64 | 5000        | The listed amount of the loan applied for by the borrower.                                                                        |
| funded_amnt         | float64 | 5000        | The total amount committed to that loan at that point in time.                                                                    |
| funded_amnt_inv     | float64 | 49750       | The total amount committed by investors for that loan at that point in time.                                                      |
| term                | object  | 36 months   | The number of payments on the loan. Values are in months and can be either 36 or 60.                                              |
| int_rate            | object  | 10.65%      | Interest Rate on the loan                                                                                                         |
| installment         | float64 | 162.87      | The monthly payment owed by the borrower if the loan originates.                                                                  |
| grade               | object  | B           | LC assigned loan grade                                                                                                            |
| sub_grade           | object  | B2          | LC assigned loan subgrade                                                                                                         |
| emp_title           | object  | NaN         | The job title supplied by the Borrower when applying for the loan.                                                                |
| emp_length          | object  | 10+ years   | Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years. |
| home_ownership      | object  | RENT        | The home ownership status provided by the borrower during registration. Our values are: RENT, OWN, MORTGAGE, OTHER.               |
| annual_inc          | float64 | 24000       | The self-reported annual income provided by the borrower during registration.                                                     |
| verification_status | object  | Verified    | Indicates if income was verified by LC, not verified, or if the income source was verified                                        |
| issue_d             | object  | Dec-2011    | The month which the loan was funded                                                                                               |
| loan_status         | object  | Charged Off | Current status of the loan                                                                                                        |
| pymnt_plan          | object  | n           | Indicates if a payment plan has been put in place for the loan                                                                    |
| purpose             | object  | car         | A category provided by the borrower for the loan request.                                                                         |

After analyzing each column, we can conclude that the following features need to be removed:

- id: randomly generated field by Lending Club for unique identification purposes only
- member_id: also a randomly generated field by Lending Club for unique identification purposes only
- funded_amnt: leaks data from the future (after the loan is already started to be funded)
- funded_amnt_inv: also leaks data from the future (after the loan is already started to be funded)
- grade: contains redundant information as the interest rate column (int_rate)
- sub_grade: also contains redundant information as the interest rate column (int_rate)
- emp_title: requires other data and a lot of processing to potentially be useful
- issue_d: leaks data from the future (after the loan is already completely funded)

In [2]:
drop_columns = ['id','member_id','funded_amnt','funded_amnt_inv','grade','sub_grade','emp_title','issue_d']
loans_2007 = loans_2007.drop(drop_columns, axis = 1)

### Columns group 2 of 3 

| name                | dtype   | first value | description                                                                                                                                                                                              |
|---------------------|---------|-------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| title               | object  | Computer    | The loan title provided by the borrower                                                                                                                                                                  |
| zip_code            | object  | 860xx       | The first 3 numbers of the zip code provided by the borrower in the loan application.                                                                                                                    |
| addr_state          | object  | AZ          | The state provided by the borrower in the loan application                                                                                                                                               |
| dti                 | float64 | 27.65       | A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income. |
| delinq_2yrs         | float64 | 0           | The number of 30+ days past-due incidences of delinquency in the borrower's credit file for the past 2 years                                                                                             |
| earliest_cr_line    | object  | Jan-1985    | The month the borrower's earliest reported credit line was opened                                                                                                                                        |
| inq_last_6mths      | float64 | 1           | The number of inquiries in past 6 months (excluding auto and mortgage inquiries)                                                                                                                         |
| open_acc            | float64 | 3           | The number of open credit lines in the borrower's credit file.                                                                                                                                           |
| pub_rec             | float64 | 0           | Number of derogatory public records                                                                                                                                                                      |
| revol_bal           | float64 | 13648       | Total credit revolving balance                                                                                                                                                                           |
| revol_util          | object  | 83.7%       | Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit.                                                                               |
| total_acc           | float64 | 9           | The total number of credit lines currently in the borrower's credit file                                                                                                                                 |
| initial_list_status | object  | f           | The initial listing status of the loan. Possible values are – W, F                                                                                                                                       |
| out_prncp           | float64 | 0           | Remaining outstanding principal for total amount funded                                                                                                                                                  |
| out_prncp_inv       | float64 | 0           | Remaining outstanding principal for portion of total amount funded by investors                                                                                                                          |
| total_pymnt         | float64 | 5863.16     | Payments received to date for total amount funded                                                                                                                                                        |
| total_pymnt_inv     | float64 | 5833.84     | Payments received to date for portion of total amount funded by investors                                                                                                                                |
| total_rec_prncp     | float64 | 5000        | Principal received to date                                                                                                                                                                               |

Within this group of columns, we need to drop the following columns:

- zip_code: redundant with the addr_state column since only the first 3 digits of the 5-digit zip code are visible (which can only be used to identify the state the borrower lives in)
- out_prncp: leaks data from the future, (after the loan already started to be paid off)
- out_prncp_inv: also leaks data from the future, (after the loan already started to be paid off)
- total_pymnt: also leaks data from the future, (after the loan already started to be paid off)
- total_pymnt_inv: also leaks data from the future, (after the loan already started to be paid off)
- total_rec_prncp: also leaks data from the future, (after the loan already started to be paid off)

In [3]:
len(loans_2007['title'].unique())

21265

In [4]:
drop_columns_2 = ['zip_code','out_prncp','out_prncp_inv','total_pymnt','total_pymnt_inv','total_rec_prncp']
loans_2007 = loans_2007.drop(drop_columns_2, axis = 1)

### Columns group 3 of 3 

| name                       | dtype   | first value | description                                                                                          |
|----------------------------|---------|-------------|------------------------------------------------------------------------------------------------------|
| total_rec_int              | float64 | 863.16      | Interest received to date                                                                            |
| total_rec_late_fee         | float64 | 0           | Late fees received to date                                                                           |
| recoveries                 | float64 | 0           | post charge off gross recovery                                                                       |
| collection_recovery_fee    | float64 | 0           | post charge off collection fee                                                                       |
| last_pymnt_d               | object  | Jan-2015    | Last month payment was received                                                                      |
| last_pymnt_amnt            | float64 | 171.62      | Last total payment amount received                                                                   |
| last_credit_pull_d         | object  | Jun-2016    | The most recent month LC pulled credit for this loan                                                 |
| collections_12_mths_ex_med | float64 | 0           | Number of collections in 12 months excluding medical collections                                     |
| policy_code                | float64 | 1           | publicly available policy_code=1 new products not publicly available policy_code=2                   |
| application_type           | object  | INDIVIDUAL  | Indicates whether the loan is an individual application or a joint application with two co-borrowers |
| acc_now_delinq             | float64 | 0           | The number of accounts on which the borrower is now delinquent.                                      |
| chargeoff_within_12_mths   | float64 | 0           | Number of charge-offs within 12 months                                                               |
| delinq_amnt                | float64 | 0           | The past-due amount owed for the accounts on which the borrower is now delinquent.                   |
| pub_rec_bankruptcies       | float64 | 0           | Number of public record bankruptcies                                                                 |
| tax_liens                  | float64 | 0           | Number of tax liens                                                                                  |

In the last group of columns, we need to drop the following columns:

- total_rec_int: leaks data from the future, (after the loan has started to be paid off),
- total_rec_late_fee: leaks data from the future, (after the loan has started to be paid off),
- recoveries: leaks data from the future, (after the loan has started to be paid off),
- collection_recovery_fee: leaks data from the future, (after the loan has started to be paid off),
- last_pymnt_d: leaks data from the future, (after the loan has started to be paid off),
- last_pymnt_amnt: leaks data from the future, (after the loan has started to be paid off).

In [5]:
drop_columns_3 = ['total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee','last_pymnt_d','last_pymnt_amnt']
loans_2007 = loans_2007.drop(drop_columns_3, axis = 1)

### Choosing target value

The only column that directly describes if a loan was paid off on time, had delayed payments, or was defaulted on the borrower is 'loan_status'. This column should be the one used as target.

In [6]:
loans_2007['loan_status'].value_counts()

Fully Paid                                             33136
Charged Off                                             5634
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  961
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        24
In Grace Period                                           20
Late (16-30 days)                                          8
Default                                                    3
Name: loan_status, dtype: int64

| Loan Status                                         | Count | Meaning                                                                                                                                           |
|-----------------------------------------------------|-------|---------------------------------------------------------------------------------------------------------------------------------------------------|
| Fully Paid                                          | 33136 | Loan has been fully paid off.                                                                                                                     |
| Charged Off                                         | 5634  | Loan for which there is no longer a reasonable expectation of further payments.                                                                   |
| Does not meet the credit policy. Status:Fully Paid  | 1988  | While the loan was paid off, the loan application today would no longer meet the credit policy and wouldn't be approved on to the marketplace.    |
| Does not meet the credit policy. Status:Charged Off | 761   | While the loan was charged off, the loan application today would no longer meet the credit policy and wouldn't be approved on to the marketplace. |
| In Grace Period                                     | 20    | The loan is past due but still in the grace period of 15 days.                                                                                    |
| Late (16-30 days)                                   | 8     | Loan hasn't been paid in 16 to 30 days (late on the current payment).                                                                             |
| Late (31-120 days)                                  | 24    | Loan hasn't been paid in 31 to 120 days (late on the current payment).                                                                            |
| Current                                             | 961   | Loan is up to date on current payments.                                                                                                           |
| Default                                             | 3     | Loan is defaulted on and no payment has been made for more than 121 days.                                                                         |

The values that describe loans that are still ongoing are of no use. The values Fully paid and Charged Off are the ones that describe the outcome of a loan and can be used to treat the problem as a binary classification one.
The value Default is similar to Charged Off, but with the difference that there is some hope that it may be payed back at some point.

Itmust be noted that there is class imbalance within the two classes. There are 33136 fully paid off loans and 5634 charged off. This will produce a bias towards the paid off class, so this should be addressed at some point.

In [7]:
mask = (loans_2007['loan_status'] == 'Fully Paid' )| (loans_2007['loan_status'] == 'Charged Off')

loans_2007 = loans_2007[mask]

mapping_dict = {
    "loan_status": {
        "Charged Off": 0,
        "Fully Paid": 1
    }
}
loans_2007 = loans_2007.replace(mapping_dict)

In [8]:
loans_2007.head()

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,purpose,title,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,5000.0,36 months,10.65%,162.87,10+ years,RENT,24000.0,Verified,1,n,credit_card,Computer,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,2500.0,60 months,15.27%,59.83,< 1 year,RENT,30000.0,Source Verified,0,n,car,bike,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,2400.0,36 months,15.96%,84.33,10+ years,RENT,12252.0,Not Verified,1,n,small_business,real estate business,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,10000.0,36 months,13.49%,339.31,10+ years,RENT,49200.0,Source Verified,1,n,other,personel,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,f,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
5,5000.0,36 months,7.90%,156.46,3 years,RENT,36000.0,Source Verified,1,n,wedding,My wedding loan I promise to pay back,AZ,11.20,0.0,Nov-2004,3.0,9.0,0.0,7963.0,28.3%,12.0,f,Jan-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


Any column that contains only one unique value should be removed. They are not useful for the model since they don't add any information to each loan application.

In [9]:
drop_columns_4 = []

for column in loans_2007.columns:
    non_null = loans_2007[column].dropna()
    unique_non_null = non_null.unique()
    num_true_unique = len(unique_non_null)
    if num_true_unique == 1:
        drop_columns_4.append(column)
        
loans_2007 = loans_2007.drop(drop_columns_4, axis = 1)

print(drop_columns_4)

['pymnt_plan', 'initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens']


## Preparing The Features

Mathematics underlying most machine learning models assumes that the data is numerical and contains no missing values. To reinforce this requirement, scikit-learn will return an error if you try to train a model using data that contain missing values or non-numeric values when working with models like linear regression and logistic regression.

In this section the data is prepared for machine learning by focusing on handling missing values, converting categorical columns to numeric columns, and removing any other extraneous columns that are encountered throughout this process.

In [10]:
loans = loans_2007


null_counts = loans.isnull().sum()
print(null_counts)

loan_amnt                  0
term                       0
int_rate                   0
installment                0
emp_length              1036
home_ownership             0
annual_inc                 0
verification_status        0
loan_status                0
purpose                    0
title                     11
addr_state                 0
dti                        0
delinq_2yrs                0
earliest_cr_line           0
inq_last_6mths             0
open_acc                   0
pub_rec                    0
revol_bal                  0
revol_util                50
total_acc                  0
last_credit_pull_d         2
pub_rec_bankruptcies     697
dtype: int64


- emp_length              1036
- title                     11
- revol_util                50
- last_credit_pull_d         2
- pub_rec_bankruptcies     697

The column emp_length represents the employment length of the borrower, which is frequently used to assess how risky a potential loan is. So this column gives useful information and should be kept, even though the rows with missing values could be dropped.

The column pub_rec_bankruptcies has the 94% of its values in the same category, so it won't have much predictive value. Therefore, it may be completely dropped.

In [11]:
print(loans.pub_rec_bankruptcies.value_counts(normalize=True, dropna=False))

0.0    0.939438
1.0    0.042456
NaN    0.017978
2.0    0.000129
Name: pub_rec_bankruptcies, dtype: float64


In [12]:
loans = loans.drop(['pub_rec_bankruptcies'], axis = 1)
loans = loans.dropna()
print(loans.dtypes.value_counts())

object     11
float64    10
int64       1
dtype: int64


### Processing Object Type Columns

The columns with object type should be converted to numerical data types.

In [13]:
object_columns_df = loans.select_dtypes(include=['object'])

print(object_columns_df.iloc[0])

term                     36 months
int_rate                    10.65%
emp_length               10+ years
home_ownership                RENT
verification_status       Verified
purpose                credit_card
title                     Computer
addr_state                      AZ
earliest_cr_line          Jan-1985
revol_util                   83.7%
last_credit_pull_d        Jun-2016
Name: 0, dtype: object


#### Categorical Columns

- home_ownership: home ownership status, can only be 1 of 4 categorical values according to the data dictionary,
- verification_status: indicates if income was verified by Lending Club,
- emp_length: number of years the borrower was employed upon time of application,
- term: number of payments on the loan, either 36 or 60,
- addr_state: borrower's state of residence,
- purpose: a category provided by the borrower for the loan request,
- title: loan title provided by the borrower,


Based on the first row's values for purpose and title, it seems like these columns could reflect the same information. It needs exploration of the unique value counts separately to confirm if this is true.

#### Numeric Columns that need to be Converted

- int_rate: interest rate of the loan in %,
- revol_util: revolving line utilization rate or the amount of credit the borrower is using relative to all available credit.

#### Other Columns to be dropped
Some of the columns contain date values that would require a good amount of feature engineering for them to be potentially useful:

- earliest_cr_line: The month the borrower's earliest reported credit line was opened,
- last_credit_pull_d: The most recent month Lending Club pulled credit for this loan.

In [14]:
cols = ['home_ownership', 'verification_status', 'emp_length', 'term', 'addr_state']

for col in cols:
    print('-----{}-----'.format(col))
    print(loans[col].value_counts())
    print('------------')
    print('\n')

-----home_ownership-----
RENT        18112
MORTGAGE    16686
OWN          2778
OTHER          96
NONE            3
Name: home_ownership, dtype: int64
------------


-----verification_status-----
Not Verified       16281
Verified           11856
Source Verified     9538
Name: verification_status, dtype: int64
------------


-----emp_length-----
10+ years    8545
< 1 year     4513
2 years      4303
3 years      4022
4 years      3353
5 years      3202
1 year       3176
6 years      2177
7 years      1714
8 years      1442
9 years      1228
Name: emp_length, dtype: int64
------------


-----term-----
 36 months    28234
 60 months     9441
Name: term, dtype: int64
------------


-----addr_state-----
CA    6776
NY    3614
FL    2704
TX    2613
NJ    1776
IL    1447
PA    1442
VA    1347
GA    1323
MA    1272
OH    1149
MD    1008
AZ     807
WA     788
CO     748
NC     729
CT     711
MI     678
MO     648
MN     581
NV     466
SC     454
WI     427
OR     422
AL     420
LA     420
KY     3

The addr_state column contains many discrete values, and 49 dummy variable columns would need to be added to use it for classification. This would make the dataframe much larger and could slow down how quickly the code runs. This columns should be removed from consideration.

##### Exploring the difference between title and purpose columns

In [15]:
print(len(loans['title'].value_counts()))
print(len(loans['purpose'].value_counts()))

18881
14


In [16]:

print(loans['purpose'].value_counts())

debt_consolidation    17751
credit_card            4911
other                  3711
home_improvement       2808
major_purchase         2083
small_business         1719
car                    1459
wedding                 916
medical                 655
moving                  552
house                   356
vacation                348
educational             312
renewable_energy         94
Name: purpose, dtype: int64


It seems that purpose and title have overlapping information, but purpose contains few discrete values that are more useful for our classification.

##### Drop the non useful columns 

In [17]:
loans = loans.drop(['last_credit_pull_d','addr_state','title','earliest_cr_line'], axis = 1)

##### Convert int_rate and revol_util to float data type after removing the % symbol

In [18]:
loans['int_rate'] = loans['int_rate'].str.rstrip('%').astype('float')

loans['revol_util'] = loans['revol_util'].str.rstrip('%').astype('float')

##### Clean the emp_length column
Erring in the side of caution with the extreme points:

- 10+ years is categorized as 10 
- < 1 year and n/a are categorized as 0

In [19]:
mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}
loans = loans.replace(mapping_dict)

##### Encode home_ownership, verification_status, purpose, and term columns as dummy

In [20]:
dummy_df = pd.get_dummies(loans[['home_ownership','verification_status','purpose','term']])

loans = pd.concat([loans, dummy_df], axis=1)

loans = loans.drop(['home_ownership','verification_status','purpose','term'], axis=1)

In [21]:
loans.head()

,loan_amnt,int_rate,installment,emp_length,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months
0,5000.0,10.65,162.87,10,24000.0,1,27.65,0.0,1.0,3.0,0.0,13648.0,83.7,9.0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,2500.0,15.27,59.83,0,30000.0,0,1.00,0.0,5.0,3.0,0.0,1687.0,9.4,4.0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,2400.0,15.96,84.33,10,12252.0,1,8.72,0.0,2.0,2.0,0.0,2956.0,98.5,10.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
3,10000.0,13.49,339.31,10,49200.0,1,20.00,0.0,1.0,10.0,0.0,5598.0,21.0,37.0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
5,5000.0,7.90,156.46,3,36000.0,1,11.20,0.0,3.0,9.0,0.0,7963.0,28.3,12.0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0


## Making Predictions

As we prepared the data, we removed columns that had data leakage issues, contained redundant information, or required additional processing to turn into useful features. We cleaned features that had formatting issues and converted categorical columns to dummy variables.

It was noted that there's a class imbalance in our target column, loan_status. There are about 6 times as many loans that were paid off on time (positive case, label of 1) than those that weren't (negative case, label of 0). Imbalances can cause issues with many machine learning algorithms, where they appear to have high accuracy, but actually aren't learning from the training data. Due to its potential to cause issues, we need to keep the class imbalance in mind as we build machine learning models.

In [22]:
loans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37675 entries, 0 to 39785
Data columns (total 38 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   loan_amnt                            37675 non-null  float64
 1   int_rate                             37675 non-null  float64
 2   installment                          37675 non-null  float64
 3   emp_length                           37675 non-null  int64  
 4   annual_inc                           37675 non-null  float64
 5   loan_status                          37675 non-null  int64  
 6   dti                                  37675 non-null  float64
 7   delinq_2yrs                          37675 non-null  float64
 8   inq_last_6mths                       37675 non-null  float64
 9   open_acc                             37675 non-null  float64
 10  pub_rec                              37675 non-null  float64
 11  revol_bal                   

The goal is to predict if a borrower will pay off their loan on time or not. This is a binary classification problem, with the values of loan_status column being 0 or 1.

### Error Metric 

In this case, we're primarily concerned with false positives and false negatives. Both of these are different types of misclassifications. With a false positive, we predict that a loan will be paid off on time, but it actually isn't. This costs us money, since we fund loans that lose us money. With a false negative, we predict that a loan won't be paid off on time, but it actually would be paid off on time. This loses us potential money, since we didn't fund a loan that actually would have been paid off.

| loan_status | prediction | error type     |
|-------------|------------|----------------|
| 0           | 1          | False Positive |
| 1           | 1          | True Positive  |
| 0           | 0          | True Negative  |
| 1           | 0          | False Negative |

Since we're viewing this problem from the standpoint of a conservative investor, we need to treat false positives differently than false negatives. A conservative investor would want to minimize risk and avoid false positives as much as possible. They'd be more secure with missing out on opportunities (false negatives) than they would be with funding a risky loan (false positives).

Due to the class imbalance, there is a high risk of losing money. There are 6 times more paid off loans than charged off loans. This means that simply giving a number 1 to all the loans, the prediction would be 85.7% accurate.  Let's say we loan out 1000 dollars on average to each of 7 borrowers, one of them will not pay back. Each borrower pays us 10% interest back. We will make a projected profit of 100 dollars on each loan. WE end up losing 400 $.

The metric should optimize for:

- High recall (True Positive Rate): "the percentage of the loans that shouldn't be funded that I would fund".
- Low Fall-Out (False Positive Rate) : "the percentage of loans that should be funded that I would fund".

$FPR = \cfrac{False Positives}{False Positives + True Negatives}$

$TPR = \cfrac{True Positives}{True Positives + False Negatives}$

### Logistic Regression

In order to get a realistic depiction of the acccuracy of the model, k-fold cross validation is applied by means of cross_val_predict() function from the sklearn.model_selection package.

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
lr = LogisticRegression()

features = loans.drop(['loan_status'], axis = 1)
target = loans['loan_status']

predictions = cross_val_predict(lr, features, target, cv=3)

predictions = pd.Series(predictions)

tp_filter = (predictions == 1) & (loans['loan_status'] == 1)
fn_filter = (predictions == 0) & (loans['loan_status'] == 1)
fp_filter = (predictions == 1) & (loans['loan_status'] == 0)
tn_filter = (predictions == 0) & (loans['loan_status'] == 0)
tp = len(loans[tp_filter])
fn = len(loans[fn_filter])
fp = len(loans[fp_filter])
tn = len(loans[tn_filter])

tpr = tp / (tp + fn) 

fpr = fp / (fp + tn)

C:\Users\adrig\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\adrig\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

In [24]:
print('TPR: ', tpr)
print('FPR: ', fpr)

TPR:  0.9983613004719455
FPR:  0.9992102665350444


#### Imbalance in the classes

The TPR and FPR values indiucate that the imbalance of classes makes the classifier to maximice accuracy by predicting with the value of the most common class. These values are what we would expect if the model was predicting all ones since almost all the predictions that should be negative are false postitives (FPR is almost equal to 1).

One way to solve thius is by penalizing the misclassification of the minority class during the training process (setting parameter class_weight = 'balanced').

By setting the class_weight parameter to balanced, the penalty is set to be inversely proportional to the class frequencies. This would mean that for the classifier, correctly classifying a row where loan_status is 0 is 6 times more important than correctly classifying a row where loan_status is 1.

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict

lr = LogisticRegression(class_weight = 'balanced')


predictions = cross_val_predict(lr, features, target, cv=3)

#Converting to Series objects let's take advantage of boolean filtering and arithmetic operations from pandas
predictions = pd.Series(predictions)


tp_filter = (predictions == 1) & (loans['loan_status'] == 1)
fn_filter = (predictions == 0) & (loans['loan_status'] == 1)
fp_filter = (predictions == 1) & (loans['loan_status'] == 0)
tn_filter = (predictions == 0) & (loans['loan_status'] == 0)
tp = len(loans[tp_filter])
fn = len(loans[fn_filter])
fp = len(loans[fp_filter])
tn = len(loans[tn_filter])

tpr = tp / (tp + fn) 

fpr = fp / (fp + tn)

print('TPR: ', tpr)
print('FPR: ', fpr)

C:\Users\adrig\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\adrig\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

TPR:  0.5465718405873099
FPR:  0.5237907206317868


C:\Users\adrig\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\adrig\anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\adrig\anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\adrig\anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\adrig\anaconda3\lib\site-

From a conservative investor's standpoint, it's reassuring that the false positive rate is lower, because it means that we'll be able to do a better job at avoiding bad loans than if we funded everything.

We can also specify a penalty manually if we want to adjust the rates more. To do this, we need to pass in a dictionary of penalty values to the class_weight parameter.

Note that this comes at the expense of true positive rate. While we have fewer false positives, we're also missing opportunities to fund more loans and potentially make more money. Given that we're approaching this as a conservative investor, this strategy makes sense, but it's worth keeping in mind the tradeoffs.

In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict

penalty = {
    0: 10,
    1: 1
    
}

lr = LogisticRegression(class_weight = penalty)


predictions = cross_val_predict(lr, features, target, cv=3)

#Converting to Series objects let's take advantage of boolean filtering and arithmetic operations from pandas
predictions = pd.Series(predictions)


tp_filter = (predictions == 1) & (loans['loan_status'] == 1)
fn_filter = (predictions == 0) & (loans['loan_status'] == 1)
fp_filter = (predictions == 1) & (loans['loan_status'] == 0)
tn_filter = (predictions == 0) & (loans['loan_status'] == 0)
tp = len(loans[tp_filter])
fn = len(loans[fn_filter])
fp = len(loans[fp_filter])
tn = len(loans[tn_filter])

tpr = tp / (tp + fn) 

fpr = fp / (fp + tn)

print('TPR: ', tpr)
print('FPR: ', fpr)

C:\Users\adrig\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


TPR:  0.16996591504981648
FPR:  0.16663376110562686


C:\Users\adrig\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\adrig\anaconda3\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\adrig\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\adrig\anaconda3\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\adrig\anaconda3\lib\site-

### Random Forest Classifier

In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict


lr = RandomForestClassifier(class_weight = 'balanced', random_state = 1)


predictions = cross_val_predict(lr, features, target, cv=3)

#Converting to Series objects let's take advantage of boolean filtering and arithmetic operations from pandas
predictions = pd.Series(predictions)


tp_filter = (predictions == 1) & (loans['loan_status'] == 1)
fn_filter = (predictions == 0) & (loans['loan_status'] == 1)
fp_filter = (predictions == 1) & (loans['loan_status'] == 0)
tn_filter = (predictions == 0) & (loans['loan_status'] == 0)
tp = len(loans[tp_filter])
fn = len(loans[fn_filter])
fp = len(loans[fp_filter])
tn = len(loans[tn_filter])

tpr = tp / (tp + fn) 

fpr = fp / (fp + tn)

print('TPR: ', tpr)
print('FPR: ', fpr)

TPR:  0.996722600943891
FPR:  0.9982230997038499


C:\Users\adrig\anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\adrig\anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\adrig\anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\adrig\anaconda3\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


Unfortunately, using a random forest classifier didn't improve our false positive rate. The model is likely too heavy on the 1 class, and still mostly predicting 1s. We could fix this by applying a harsher penalty for misclassifications of 0s.

In [46]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict

penalty = {
    0: 100,
    1: 1
    
}

lr = RandomForestClassifier(class_weight = penalty, random_state = 1, max_depth = 30, min_samples_leaf= 10, min_samples_split = 50)


predictions = cross_val_predict(lr, features, target, cv=3)

#Converting to Series objects let's take advantage of boolean filtering and arithmetic operations from pandas
predictions = pd.Series(predictions)


tp_filter = (predictions == 1) & (loans['loan_status'] == 1)
fn_filter = (predictions == 0) & (loans['loan_status'] == 1)
fp_filter = (predictions == 1) & (loans['loan_status'] == 0)
tn_filter = (predictions == 0) & (loans['loan_status'] == 0)
tp = len(loans[tp_filter])
fn = len(loans[fn_filter])
fp = len(loans[fp_filter])
tn = len(loans[tn_filter])

tpr = tp / (tp + fn) 

fpr = fp / (fp + tn)

print('TPR: ', tpr)
print('FPR: ', fpr)

TPR:  0.25829181961195596
FPR:  0.25923000987166833


C:\Users\adrig\anaconda3\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\adrig\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\adrig\anaconda3\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\adrig\anaconda3\lib\site-packages\ipykernel_launcher.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


If we had randomly picked loans to fund, borrowers would have defaulted on 14.5% of them, and none of our models is not better than that, although we're excluding more loans than a random strategy would. Given this, there's still quite a bit of room to improve:

- We can tweak the penalties further.
- We can try models other than a random forest and logistic regression.
- We can use some of the columns we discarded to generate better features.
- We can ensemble multiple models to get more accurate predictions.
- We can tune the parameters of the algorithm to achieve higher performance.